In [5]:
import pandas as pd
import numpy as np
#from fuzzywuzzy import process
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import joblib
import time

In [6]:
animes = pd.read_csv('anime.csv')
ratings = pd.read_csv('rating.csv')

In [7]:
animes

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [8]:
animes.anime_id.nunique()

12294

In [9]:
animes = animes[animes['type'] == 'TV']
animes

,anime_id,name,genre,type,episodes,rating,members
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
...,...,...,...,...,...,...,...
11104,34522,"Wake Up, Girls! Shin Shou","Drama, Music",TV,Unknown,NaN,381
11106,34467,Yami Shibai 4th Season,"Dementia, Horror, Supernatural",TV,Unknown,NaN,1838
11107,32615,Youjo Senki,"Magic, Military",TV,Unknown,NaN,6652
11110,34284,Yuuki Yuuna wa Yuusha de Aru: Washio Sumi no Shou,"Drama, Fantasy, Magic, Slice of Life",TV,6,NaN,2593


In [10]:
ratings.rating.replace({-1: 0}, inplace = True)
ratings.dropna(axis = 0, inplace=True)

In [11]:
ratings.anime_id.nunique()

11200

In [12]:
#ratings = ratings.groupby('anime_id').filter(lambda x : (x['anime_id'].count()>=100).any())
#ratings

In [13]:
#ratings = ratings.groupby('user_id').filter(lambda x : (x['user_id'].count()>=100).any())
#ratings

In [14]:
user_counts = ratings.groupby('user_id').count().sort_values(by='rating', ascending=False)
user_counts

,anime_id,rating
user_id,,
48766,10227,10227
42635,3747,3747
53698,2905,2905
57620,2702,2702
59643,2633,2633
...,...,...
11323,1,1
48775,1,1
11328,1,1


In [15]:
top_users = list(user_counts.head(10000).index)

len(top_users)

10000

In [16]:
ratings = ratings[ratings['user_id'].isin(top_users)]
ratings

,user_id,anime_id,rating
302,5,6,8
303,5,15,6
304,5,17,6
305,5,18,6
306,5,20,6
...,...,...,...
7813330,73507,8231,5
7813331,73507,8348,5
7813332,73507,8440,7
7813333,73507,8769,8


In [17]:
animes.to_csv('animes_preprocessed.csv', index=False)
ratings.to_csv('ratings_preprocessed.csv', index=False)

In [18]:
combined = pd.merge(animes, ratings, on='anime_id')
combined

,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
0,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,17,10
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,38,10
2,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,43,4
3,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,46,9
4,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,129,9
...,...,...,...,...,...,...,...,...,...
2521252,7808,Zukkoke Knight: Don De La Mancha,"Adventure, Comedy, Historical, Romance",TV,23,6.47,172,1822,6
2521253,7808,Zukkoke Knight: Don De La Mancha,"Adventure, Comedy, Historical, Romance",TV,23,6.47,172,48766,0
2521254,7808,Zukkoke Knight: Don De La Mancha,"Adventure, Comedy, Historical, Romance",TV,23,6.47,172,49127,6
2521255,7808,Zukkoke Knight: Don De La Mancha,"Adventure, Comedy, Historical, Romance",TV,23,6.47,172,51693,7


In [19]:
pivot = combined.pivot_table(index='name',columns='user_id',values='rating_y').fillna(0)
pivot

user_id,5,7,17,38,43,46,54,123,129,139,...,73417,73422,73457,73460,73476,73491,73499,73502,73503,73507
name,,,,,,,,,,,,,,,,,,,,,
.hack//Roots,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Sign,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
.hack//Tasogare no Udewa Densetsu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0-sen Hayato,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
009-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
gdgd Fairies 2,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
iDOLM@STER Xenoglossia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s.CRY.ed,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
anime_titles = pivot.reset_index()
anime_titles = anime_titles['name'].to_frame()
anime_titles

,name
0,.hack//Roots
1,.hack//Sign
2,.hack//Tasogare no Udewa Densetsu
3,0-sen Hayato
4,009-1
...,...
3394,gdgd Fairies 2
3395,iDOLM@STER Xenoglossia
3396,s.CRY.ed
3397,xxxHOLiC


In [19]:
anime_titles.to_csv('anime_titles.csv', index=False)

In [23]:
def get_index(anime_name):
    record = anime_titles[anime_titles['name'] == anime_name]
    anime_titles = list(record.index) 
    return anime_titles[0]

In [26]:
record = anime_titles[anime_titles['name'] == 'Boku no Hero Academia 2nd Season']
record

,name


In [24]:
get_index('Boku no Hero Academia 2nd Season')

UnboundLocalError: local variable 'anime_titles' referenced before assignment

In [21]:
matrix = csr_matrix(pivot.values)

In [22]:
import joblib

filename = 'matrix.sav'
joblib.dump(matrix, filename)

['matrix.sav']

In [23]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_jobs=-1)
model_knn.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [24]:
filename = 'model_knn.sav'
joblib.dump(model_knn, filename)

['model_knn.sav']

In [25]:
#similarities = cosine_similarity(matrix,dense_output=False)
#print(similarities)

In [26]:
def recommend():
    name = input("Enter an anime")

    idx=process.extractOne(name, anime_titles['name'])

    distances, indices = model_knn.kneighbors(pivot.iloc[idx[2],:].values.reshape(1, -1), n_neighbors = 20)
    
    print(idx)
    
    names = []
    distances = distances.flatten()
    
    for i in range(0, len(distances)):
        if i == 0:
            print('Recommendations for {0}:\n'.format(pivot.index[idx[2]]))
        else:
            names.append(pivot.index[indices.flatten()[i]])


    similar_anime = pd.DataFrame()
    similar_anime['Similar Anime'] = names
    similar_anime['distances'] = distances[1:len(distances)]
    return similar_anime

In [27]:
yo = recommend()
yo

Enter an animedragon ball


NameError: name 'process' is not defined